In [1]:
import json
import os
from openai import OpenAI
from tqdm.auto import tqdm
from IPython.display import clear_output
import pandas as pd

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
docs = pd.read_csv("data_acquisition/raw_data.csv")
docs.head()

,doc,index,type,content
0,CELEX_32019R2019,0,table_row,5.12.2019 EN Official Journal of the European ...
1,CELEX_32019R2019,1,title,COMMISSION REGULATION (EU) 2019/2019
2,CELEX_32019R2019,2,title,laying down ecodesign requirements for refrige...
3,CELEX_32019R2019,3,title,(Text with EEA relevance)
4,CELEX_32019R2019,4,text,"THE EUROPEAN COMMISSION,"


In [ ]:
dates = {}

In [ ]:
j = 0
for i, row in docs[docs["index"] == 0].iterrows():
    j += 1
    document = row['doc']
    content = str(row["content"])
    splt = content.split(' ')
    if len(splt) > 1:
        date = splt[0]
        dates[document] = date
    else:
        sel = docs[docs["doc"] == document]
        line = sel[sel['index'] == 1]['content'].values[0]
        date = completion = client.chat.completions.create(model="gpt-3.5-turbo",
                                                   messages=[
                                                       {"role": "system", "content": "You are a simple helper whose job it is to extract the date of an article from a headline. You are only supposed to return the date in the format dd.mm.yyyy.\n \
                                                        Example:\n \
                                                        Headline: 94/833/Euratom: Opinion of the Commission of 14 December 1994 concerning the plan for the disposal of radioactive waste from the Chooz B nuclear power station\n \
                                                        Output: 14.12.1994\n\
                                                        Pad the date with zeros if necessary."},
                                                       {"role": "user", "content": f"Headline: {line}\nOutput:"}
                                                   ]).choices[0].message.content
        match = re.fullmatch(r'(0[1-9]|[12][0-9]|3[01])\.(0[1-9]|1[012])\.(19|20)\d\d', str(date))
        dates[document] = match.string
    clear_output(wait=True)
    print(f"processed {j} documents")

processed 65252 documents


In [ ]:
with open("dates.json", "w") as file:
    json.dump(dates, file)
dates